In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 20796 (delta 32), reused 41 (delta 20), pack-reused 20737
Receiving objects: 100% (20796/20796), 16.55 MiB | 12.54 MiB/s, done.
Resolving deltas: 100% (15176/15176), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
import os

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=f0d6227b4b4bc9ec485284184bc71fdcf4abfd06fda5ee542bcc314d3f4f67a5
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import math
from math import radians
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [42]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [43]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [44]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [45]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [46]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [47]:
def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [48]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [49]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [50]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [51]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [52]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [170]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0.09:
        dataset.iloc[i,j] = 0
  return dataset

def not_dec(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns) - 1):
      K = dataset.iloc[i,j]
      T = dataset.iloc[i,j+1]
      if K > 0 and T == 0:
        dataset.iloc[i,j] = 0

In [54]:
def wind_and_hot(wind, hot):
  h = 743 + 176.5 * hot + 3.562 * wind - 13.14 * hot**2 -0.7466 * hot * wind - 0.151 * wind ** 2
  return h

In [55]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [56]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [57]:
dataset4.rename(columns = {'T' : 'Temp'},inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [58]:
dataset5= dataset4.copy()

dataset5['WST'] = dataset5.apply(lambda x: wind_and_hot(x.WS, x.Temp), axis=1)
dataset5['GHI'] = dataset5.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
dataset5.iloc[50:51,:]

In [59]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,SH,sin_time,cos_time,season,Time,WST,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,-2769.9872,0.0


In [60]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset6 = dataset5[['RH', 'Temp', 'WST', 'SH', 'WS', 'GHI', 'sin_time', 'cos_time', 'TARGET']] 
dataset6.head(1)

,RH,Temp,WST,SH,WS,GHI,sin_time,cos_time,TARGET
0,69.08,-12,-3248.71795,9.0,1.5,0.0,0.000000,1.000000,0.0
1,69.06,-12,-3248.71795,9.0,1.5,0.0,0.130526,0.991445,0.0
2,71.78,-12,-3247.51264,9.0,1.6,0.0,0.258819,0.965926,0.0
3,71.75,-12,-3247.51264,9.0,1.6,0.0,0.382683,0.923880,0.0
4,75.20,-12,-3247.51264,9.0,1.6,0.0,0.500000,0.866025,0.0
5,69.29,-11,-2771.11785,9.0,1.5,0.0,0.608761,0.793353,0.0
6,72.56,-11,-2771.11785,9.0,1.5,0.0,0.707107,0.707107,0.0
7,72.55,-11,-2772.25152,9.0,1.4,0.0,0.793353,0.608761,0.0
8,74.62,-11,-2773.38821,9.0,1.3,0.0,0.866025,0.500000,0.0
9,74.61,-11,-2773.38821,9.0,1.3,0.0,0.923880,0.382683,0.0


In [61]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
scaler.fit(dataset6)
temp_X = pd.DataFrame(scaler.transform(dataset6), columns = dataset6.columns)
temp_y = pd.DataFrame(dataset6['TARGET'])

## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [62]:
dataset7 = train_to_supervised(temp_X,temp_y, n_days)
dataset7

,RH(t-144),Temp(t-144),WST(t-144),SH(t-144),WS(t-144),GHI(t-144),sin_time(t-144),cos_time(t-144),TARGET(t-144),RH(t-143),Temp(t-143),WST(t-143),SH(t-143),WS(t-143),GHI(t-143),sin_time(t-143),cos_time(t-143),TARGET(t-143),RH(t-142),Temp(t-142),WST(t-142),SH(t-142),WS(t-142),GHI(t-142),sin_time(t-142),cos_time(t-142),TARGET(t-142),RH(t-141),Temp(t-141),WST(t-141),SH(t-141),WS(t-141),GHI(t-141),sin_time(t-141),cos_time(t-141),TARGET(t-141),RH(t-140),Temp(t-140),WST(t-140),SH(t-140),...,cos_time(t-5),TARGET(t-5),RH(t-4),Temp(t-4),WST(t-4),SH(t-4),WS(t-4),GHI(t-4),sin_time(t-4),cos_time(t-4),TARGET(t-4),RH(t-3),Temp(t-3),WST(t-3),SH(t-3),WS(t-3),GHI(t-3),sin_time(t-3),cos_time(t-3),TARGET(t-3),RH(t-2),Temp(t-2),WST(t-2),SH(t-2),WS(t-2),GHI(t-2),sin_time(t-2),cos_time(t-2),TARGET(t-2),RH(t-1),Temp(t-1),WST(t-1),SH(t-1),WS(t-1),GHI(t-1),sin_time(t-1),cos_time(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
144,0.665404,0.129630,0.566595,0.083333,0.125000,0.0,0.500000,1.000000,0.0,0.665188,0.129630,0.566595,0.083333,0.125000,0.0,0.565263,0.995722,0.0,0.694622,0.129630,0.566709,0.083333,0.133333,0.0,0.629410,0.982963,0.0,0.694297,0.129630,0.566709,0.083333,0.133333,0.0,0.691342,0.961940,0.0,0.731631,0.129630,0.566709,0.083333,...,0.896677,0.0,0.773726,0.111111,0.517596,0.083333,0.033333,0.0,0.250000,0.933013,0.0,0.773618,0.111111,0.517968,0.083333,0.058333,0.0,0.308658,0.961940,0.0,0.795368,0.111111,0.518459,0.083333,0.091667,0.0,0.370590,0.982963,0.0,0.795260,0.111111,0.518825,0.083333,0.116667,0.0,0.434737,0.995722,0.0,0.0,0.0
145,0.665188,0.129630,0.566595,0.083333,0.125000,0.0,0.565263,0.995722,0.0,0.694622,0.129630,0.566709,0.083333,0.133333,0.0,0.629410,0.982963,0.0,0.694297,0.129630,0.566709,0.083333,0.133333,0.0,0.691342,0.961940,0.0,0.731631,0.129630,0.566709,0.083333,0.133333,0.0,0.750000,0.933013,0.0,0.667677,0.148148,0.611759,0.083333,...,0.933013,0.0,0.773618,0.111111,0.517968,0.083333,0.058333,0.0,0.308658,0.961940,0.0,0.795368,0.111111,0.518459,0.083333,0.091667,0.0,0.370590,0.982963,0.0,0.795260,0.111111,0.518825,0.083333,0.116667,0.0,0.434737,0.995722,0.0,0.759333,0.092593,0.469196,0.083333,0.150000,0.0,0.500000,1.000000,0.0,0.0,0.0
146,0.694622,0.129630,0.566709,0.083333,0.133333,0.0,0.629410,0.982963,0.0,0.694297,0.129630,0.566709,0.083333,0.133333,0.0,0.691342,0.961940,0.0,0.731631,0.129630,0.566709,0.083333,0.133333,0.0,0.750000,0.933013,0.0,0.667677,0.148148,0.611759,0.083333,0.125000,0.0,0.804381,0.896677,0.0,0.703062,0.148148,0.611759,0.083333,...,0.961940,0.0,0.795368,0.111111,0.518459,0.083333,0.091667,0.0,0.370590,0.982963,0.0,0.795260,0.111111,0.518825,0.083333,0.116667,0.0,0.434737,0.995722,0.0,0.759333,0.092593,0.469196,0.083333,0.150000,0.0,0.500000,1.000000,0.0,0.759442,0.111111,0.519549,0.083333,0.166667,0.0,0.565263,0.995722,0.0,0.0,0.0
147,0.694297,0.129630,0.566709,0.083333,0.133333,0.0,0.691342,0.961940,0.0,0.731631,0.129630,0.566709,0.083333,0.133333,0.0,0.750000,0.933013,0.0,0.667677,0.148148,0.611759,0.083333,0.125000,0.0,0.804381,0.896677,0.0,0.703062,0.148148,0.611759,0.083333,0.125000,0.0,0.853553,0.853553,0.0,0.702954,0.148148,0.611652,0.083333,...,0.982963,0.0,0.795260,0.111111,0.518825,0.083333,0.116667,0.0,0.434737,0.995722,0.0,0.759333,0.092593,0.469196,0.083333,0.150000,0.0,0.500000,1.000000,0.0,0.759442,0.111111,0.519549,0.083333,0.166667,0.0,0.565263,0.995722,0.0,0.758576,0.111111,0.519788,0.083333,0.183333,0.0,0.629410,0.982963,0.0,0.0,0.0
148,0.731631,0.129630,0.566709,0.083333,0.133333,0.0,0.750000,0.933013,0.0,0.667677,0.148148,0.611759,0.083333,0.125000,0.0,0.804381,0.896677,0.0,0.703062,0.148148,0.611759,0.083333,0.125000,0.0,0.853553,0.853553,0.0,0.702954,0.148148,0.611652,0.083333,0.116667,0.0,0.896677,0.804381,0.0,0.725354,0.148148,0.611544,0.083333,...,0.995722,0.0,0.759333,0.092593,0.469196,0.083333,0.150000,0.0,0.500000,1.000000,0.0,0.759442,0.111111,0.519549,0.083333,0.166667,0.0,0.565263,0.995722,0.0,0.758576,0.111111,0.519788,0.083333,0.183333,0.0,0.629410,0.982963,0.0,0.69

In [63]:
X = dataset7.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)

y_1 = pd.DataFrame(dataset7.iloc[:,-2])
y_2 = pd.DataFrame(dataset7.iloc[:,-1])
X

,RH(t-144),Temp(t-144),WST(t-144),SH(t-144),WS(t-144),GHI(t-144),sin_time(t-144),cos_time(t-144),TARGET(t-144),RH(t-143),Temp(t-143),WST(t-143),SH(t-143),WS(t-143),GHI(t-143),sin_time(t-143),cos_time(t-143),TARGET(t-143),RH(t-142),Temp(t-142),WST(t-142),SH(t-142),WS(t-142),GHI(t-142),sin_time(t-142),cos_time(t-142),TARGET(t-142),RH(t-141),Temp(t-141),WST(t-141),SH(t-141),WS(t-141),GHI(t-141),sin_time(t-141),cos_time(t-141),TARGET(t-141),RH(t-140),Temp(t-140),WST(t-140),SH(t-140),...,GHI(t-5),sin_time(t-5),cos_time(t-5),TARGET(t-5),RH(t-4),Temp(t-4),WST(t-4),SH(t-4),WS(t-4),GHI(t-4),sin_time(t-4),cos_time(t-4),TARGET(t-4),RH(t-3),Temp(t-3),WST(t-3),SH(t-3),WS(t-3),GHI(t-3),sin_time(t-3),cos_time(t-3),TARGET(t-3),RH(t-2),Temp(t-2),WST(t-2),SH(t-2),WS(t-2),GHI(t-2),sin_time(t-2),cos_time(t-2),TARGET(t-2),RH(t-1),Temp(t-1),WST(t-1),SH(t-1),WS(t-1),GHI(t-1),sin_time(t-1),cos_time(t-1),TARGET(t-1)
144,0.665404,0.129630,0.566595,0.083333,0.125000,0.0,0.500000,1.000000,0.0,0.665188,0.129630,0.566595,0.083333,0.125000,0.0,0.565263,0.995722,0.0,0.694622,0.129630,0.566709,0.083333,0.133333,0.0,0.629410,0.982963,0.0,0.694297,0.129630,0.566709,0.083333,0.133333,0.0,0.691342,0.961940,0.0,0.731631,0.129630,0.566709,0.083333,...,0.0,0.195619,0.896677,0.0,0.773726,0.111111,0.517596,0.083333,0.033333,0.0,0.250000,0.933013,0.0,0.773618,0.111111,0.517968,0.083333,0.058333,0.0,0.308658,0.961940,0.0,0.795368,0.111111,0.518459,0.083333,0.091667,0.0,0.370590,0.982963,0.0,0.795260,0.111111,0.518825,0.083333,0.116667,0.0,0.434737,0.995722,0.0
145,0.665188,0.129630,0.566595,0.083333,0.125000,0.0,0.565263,0.995722,0.0,0.694622,0.129630,0.566709,0.083333,0.133333,0.0,0.629410,0.982963,0.0,0.694297,0.129630,0.566709,0.083333,0.133333,0.0,0.691342,0.961940,0.0,0.731631,0.129630,0.566709,0.083333,0.133333,0.0,0.750000,0.933013,0.0,0.667677,0.148148,0.611759,0.083333,...,0.0,0.250000,0.933013,0.0,0.773618,0.111111,0.517968,0.083333,0.058333,0.0,0.308658,0.961940,0.0,0.795368,0.111111,0.518459,0.083333,0.091667,0.0,0.370590,0.982963,0.0,0.795260,0.111111,0.518825,0.083333,0.116667,0.0,0.434737,0.995722,0.0,0.759333,0.092593,0.469196,0.083333,0.150000,0.0,0.500000,1.000000,0.0
146,0.694622,0.129630,0.566709,0.083333,0.133333,0.0,0.629410,0.982963,0.0,0.694297,0.129630,0.566709,0.083333,0.133333,0.0,0.691342,0.961940,0.0,0.731631,0.129630,0.566709,0.083333,0.133333,0.0,0.750000,0.933013,0.0,0.667677,0.148148,0.611759,0.083333,0.125000,0.0,0.804381,0.896677,0.0,0.703062,0.148148,0.611759,0.083333,...,0.0,0.308658,0.961940,0.0,0.795368,0.111111,0.518459,0.083333,0.091667,0.0,0.370590,0.982963,0.0,0.795260,0.111111,0.518825,0.083333,0.116667,0.0,0.434737,0.995722,0.0,0.759333,0.092593,0.469196,0.083333,0.150000,0.0,0.500000,1.000000,0.0,0.759442,0.111111,0.519549,0.083333,0.166667,0.0,0.565263,0.995722,0.0
147,0.694297,0.129630,0.566709,0.083333,0.133333,0.0,0.691342,0.961940,0.0,0.731631,0.129630,0.566709,0.083333,0.133333,0.0,0.750000,0.933013,0.0,0.667677,0.148148,0.611759,0.083333,0.125000,0.0,0.804381,0.896677,0.0,0.703062,0.148148,0.611759,0.083333,0.125000,0.0,0.853553,0.853553,0.0,0.702954,0.148148,0.611652,0.083333,...,0.0,0.370590,0.982963,0.0,0.795260,0.111111,0.518825,0.083333,0.116667,0.0,0.434737,0.995722,0.0,0.759333,0.092593,0.469196,0.083333,0.150000,0.0,0.500000,1.000000,0.0,0.759442,0.111111,0.519549,0.083333,0.166667,0.0,0.565263,0.995722,0.0,0.758576,0.111111,0.519788,0.083333,0.183333,0.0,0.629410,0.982963,0.0
148,0.731631,0.129630,0.566709,0.083333,0.133333,0.0,0.750000,0.933013,0.0,0.667677,0.148148,0.611759,0.083333,0.125000,0.0,0.804381,0.896677,0.0,0.703062,0.148148,0.611759,0.083333,0.125000,0.0,0.853553,0.853553,0.0,0.702954,0.148148,0.611652,0.083333,0.116667,0.0,0.896677,0.804381,0.0,0.725354,0.148148,0.611544,0.083333,...,0.0,0.434737,0.995722,0.0,0.759333,0.092593,0.469196,0.083333,0.150000,0.0,0.500000,1.000000,0.0,0.759442,0.111111,0.519549,0.083333,0.166667,0.0,0.565263,0.995722,0.0,0.758576,0.111111,0.519788,0.083333,0.183333,0.

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [75]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'
    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
    testset4.rename(columns = {'T' : 'Temp'},inplace = True)

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset5['WST'] = testset4.apply(lambda x: wind_and_hot(x.WS, x.Temp), axis=1)
    testset6 = testset5[['RH', 'Temp', 'WST', 'SH', 'WS', 'GHI', 'sin_time', 'cos_time', 'TARGET']] #'DHI', 'DNI', 'season', 'WS', 'SH'
    testset6 = pd.DataFrame(scaler.transform(testset6), columns=testset6.columns)
    testset7 = test_to_supervised(testset6, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset7.iloc[-48:,:]

    df_test.append(testset9)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,RH(t-144),Temp(t-144),WST(t-144),SH(t-144),WS(t-144),GHI(t-144),sin_time(t-144),cos_time(t-144),TARGET(t-144),RH(t-143),Temp(t-143),WST(t-143),SH(t-143),WS(t-143),GHI(t-143),sin_time(t-143),cos_time(t-143),TARGET(t-143),RH(t-142),Temp(t-142),WST(t-142),SH(t-142),WS(t-142),GHI(t-142),sin_time(t-142),cos_time(t-142),TARGET(t-142),RH(t-141),Temp(t-141),WST(t-141),SH(t-141),WS(t-141),GHI(t-141),sin_time(t-141),cos_time(t-141),TARGET(t-141),RH(t-140),Temp(t-140),WST(t-140),SH(t-140),...,GHI(t-5),sin_time(t-5),cos_time(t-5),TARGET(t-5),RH(t-4),Temp(t-4),WST(t-4),SH(t-4),WS(t-4),GHI(t-4),sin_time(t-4),cos_time(t-4),TARGET(t-4),RH(t-3),Temp(t-3),WST(t-3),SH(t-3),WS(t-3),GHI(t-3),sin_time(t-3),cos_time(t-3),TARGET(t-3),RH(t-2),Temp(t-2),WST(t-2),SH(t-2),WS(t-2),GHI(t-2),sin_time(t-2),cos_time(t-2),TARGET(t-2),RH(t-1),Temp(t-1),WST(t-1),SH(t-1),WS(t-1),GHI(t-1),sin_time(t-1),cos_time(t-1),TARGET(t-1)
288,0.557299,0.353704,0.946070,0.166667,0.091667,0.0,0.500000,1.000000,0.0,0.580132,0.344444,0.937611,0.166667,0.100000,0.0,0.565263,0.995722,0.0,0.629477,0.337037,0.930408,0.166667,0.108333,0.0,0.629410,0.982963,0.0,0.634672,0.335185,0.928573,0.166667,0.116667,0.0,0.691342,0.961940,0.0,0.672113,0.333333,0.926714,0.166667,...,0.000000,0.195619,0.896677,0.000000,0.805216,0.303704,0.892529,0.166667,0.041667,0.000000,0.250000,0.933013,0.000000,0.805216,0.303704,0.892580,0.166667,0.050000,0.0,0.308658,0.961940,0.0,0.798507,0.303704,0.892630,0.166667,0.058333,0.0,0.370590,0.982963,0.0,0.805108,0.301852,0.890358,0.166667,0.066667,0.0,0.434737,0.995722,0.0
289,0.580132,0.344444,0.937611,0.166667,0.100000,0.0,0.565263,0.995722,0.0,0.629477,0.337037,0.930408,0.166667,0.108333,0.0,0.629410,0.982963,0.0,0.634672,0.335185,0.928573,0.166667,0.116667,0.0,0.691342,0.961940,0.0,0.672113,0.333333,0.926714,0.166667,0.125000,0.0,0.750000,0.933013,0.0,0.672113,0.333333,0.926750,0.166667,...,0.000000,0.250000,0.933013,0.000000,0.805216,0.303704,0.892580,0.166667,0.050000,0.000000,0.308658,0.961940,0.000000,0.798507,0.303704,0.892630,0.166667,0.058333,0.0,0.370590,0.982963,0.0,0.805108,0.301852,0.890358,0.166667,0.066667,0.0,0.434737,0.995722,0.0,0.793529,0.300000,0.888011,0.166667,0.066667,0.0,0.500000,1.000000,0.0
290,0.629477,0.337037,0.930408,0.166667,0.108333,0.0,0.629410,0.982963,0.0,0.634672,0.335185,0.928573,0.166667,0.116667,0.0,0.691342,0.961940,0.0,0.672113,0.333333,0.926714,0.166667,0.125000,0.0,0.750000,0.933013,0.0,0.672113,0.333333,0.926750,0.166667,0.133333,0.0,0.804381,0.896677,0.0,0.655557,0.333333,0.926786,0.166667,...,0.000000,0.308658,0.961940,0.000000,0.798507,0.303704,0.892630,0.166667,0.058333,0.000000,0.370590,0.982963,0.000000,0.805108,0.301852,0.890358,0.166667,0.066667,0.0,0.434737,0.995722,0.0,0.793529,0.300000,0.888011,0.166667,0.066667,0.0,0.500000,1.000000,0.0,0.800130,0.298148,0.885691,0.166667,0.075000,0.0,0.565263,0.995722,0.0
291,0.634672,0.335185,0.928573,0.166667,0.116667,0.0,0.691342,0.961940,0.0,0.672113,0.333333,0.926714,0.166667,0.125000,0.0,0.750000,0.933013,0.0,0.672113,0.333333,0.926750,0.166667,0.133333,0.0,0.804381,0.896677,0.0,0.655557,0.333333,0.926786,0.166667,0.141667,0.0,0.853553,0.853553,0.0,0.650254,0.335185,0.928714,0.166667,...,0.000000,0.370590,0.982963,0.000000,0.805108,0.301852,0.890358,0.166667,0.066667,0.000000,0.434737,0.995722,0.000000,0.793529,0.300000,0.888011,0.166667,0.066667,0.0,0.500000,1.000000,0.0,0.800130,0.298148,0.885691,0.166667,0.075000,0.0,0.565263,0.995722,0.0,0.782599,0.296296,0.883348,0.166667,0.083333,0.0,0.629410,0.982963,0.0
292,0.672113,0.333333,0.926714,0.166667,0.125000,0.0,0.750000,0.933013,0.0,0.672113,0.333333,0.926750,0.166667,0.133333,0.0,0.804381,0.896677,0.0,0.655557,0.333333,0.926786,0.166667,0.141667,0.0,0.853553,0.853553,0.0,0.650254,0.335185,0.928714,0.166667,0.150000,0.0,0.896677,0.804381,0.0,0.613678,0.338889,0.932491,0.166667,...,0.000000,0.434737,0.995722,0.000000,0.793529,0.300000,0.888011,0.166667,0.066667,0.000000,0.500000,1.000000,0.000000,0.800130,0.298148,

In [100]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

k=0
quantiles = [0.1-k, 0.2-k, 0.3-k, 0.4-k, 0.5, 0.6-k, 0.7-k, 0.8-k, 0.9-k]

In [101]:
params = {}
params['objective'] = 'quantile'
params['min_child_samples'] = 10
params['min_child_weight'] = 3
params['min_split_gain'] = 0
params['bagging_fraction'] = 0.9
params['feature_fraction'] = 0.9
params['learning_rate'] = 0.007
params['save_binary'] = True
params['is_unbalance'] = False
params['subsample'] = 0.9
params['max_drop'] = 50
params['max_bin'] = 256
params['drop_rate'] = 0.1
params['num_leaves'] = 40
params['device'] = 'gpu'
params['max_depth'] = 25
params['boosting_type'] = "gbdt"
params['Min_data_in_leaf'] = 40

In [102]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
#def LGBM(q, X, Y, X_test):

def LGBM(q, X_train, X_valid, Y_train, Y_valid, X_test, estimators):    
    # (a) 모델링

    params['n_estimators'] = estimators
    model = LGBMRegressor(**params, alpha=q, random_state = 42)          
    
 #   splits = 3
 #   kfold = KFold(n_splits = splits, shuffle = True, random_state = 42)
 #   for n_fold, (trn_idx, val_idx) in enumerate(kfold.split(X)):
 #     X_train, Y_train = X.iloc[trn_idx], Y.iloc[trn_idx]
 #     X_valid, Y_valid = X.iloc[val_idx], Y.iloc[val_idx]
 
#    rnd_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=100, cv=4, scoring='neg_mean_squared_error', random_state=42)
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [103]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

#### Feature_selection 모델에 사용할 estimator 개수 ####
    fs_estimator = 2000
#### 실제 학습 모델에 사용할 n_estimator 개수 ####
    estimator = 30000


    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q,'  ########################################### 요인 선정 중 ##############################################')
        # 요인 선정은 오래 돌릴 필요 없으니까... 1000만 주었음.
        temp_p, temp_m = LGBM(q, X_train, X_valid, Y_train, Y_valid, X_test, fs_estimator)
        selected = feature_selection(temp_m, X_train)

        print(q, '  ###################################### {0}개 중 {1}개 선정 완료 ##############################################'.format(len(X_train.columns), len(selected)))
        pred, model = LGBM(q, X_train[selected], X_valid[selected], Y_train, Y_valid, X_test[selected], estimator)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [104]:
# 중요도를 %로 전환하여 반환.
# 정확도의 총 합이 "accuracy" 되는 feature들 내림차순으로 반환
# 지금은 97 이므로, 97%의 중요도 변수들은 반환하고, 3%를 차지하는 변수들은 중요하지 않다고 판단하여 버린다.

# corrs 는 어느정도 상관계수 이상을 버릴 것인지 정의하는 인자
# 여기서는 일단 0.95로 설정

accuracy = 97
corrs = 0.95

def feature_selection(models, input_data):
  data = pd.DataFrame(models.feature_importances_, index = input_data.columns, columns= ['Importance'])
  temp_sum = sum(data.Importance)
  data['Percentage'] = (data.Importance/temp_sum) * 100
  data = data.sort_values(by = 'Percentage', ascending=False)

  total_imp = 0
  for i in range(data.shape[0]):
    imp = data.iloc[i,-1]
    total_imp += imp
    if total_imp >= accuracy:
      break
  selected_list = list(data.index[:i])
  selected_data = input_data[selected_list]

##### 이 위 까지는 97%의 정확도를 가지는 특징을 잡아주는 함수, 아래는 상관계수(corr)가 0.95 이상인 변수를 제외하는 함수
  corr_matrix = selected_data.corr().abs()
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
  high_cor = [column for column in upper.columns if any(upper[column] > corrs)]
  features = [i for i in selected_data.columns if i not in high_cor]

  return features

In [105]:
tick = time.time()

# Target1
models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)

#models_1, results_1 = train_data(X, y_1, X_test)
results_1.sort_index()[:48]

0.1   ########################################### 요인 선정 중 ##############################################
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.37983
[1000]	valid_0's quantile: 1.34326
[1500]	valid_0's quantile: 1.33563
[2000]	valid_0's quantile: 1.32854
Did not meet early stopping. Best iteration is:
[2000]	valid_0's quantile: 1.32854
0.1   ###################################### 1296개 중 348개 선정 완료 ##############################################
[LightGBM] [Warning] Unknown parameter: Min_data_in_leaf
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.38022
[1000]	valid_0's quantile: 1.34591
[1500]	valid_0's quantile: 1.33559
[2000]	valid_0's quantile: 1.32988
[

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,-0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,-0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,-0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [106]:
# Target2
models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)

#models_2, results_2 = train_data(X, y_2, X_test)
tock = time.time()
results_2.sort_index()

0.1   ########################################### 요인 선정 중 ##############################################
[LightGBM] [Warning] Unknown parameter: Min_data_in_leaf
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.40403
[1000]	valid_0's quantile: 1.37448
[1500]	valid_0's quantile: 1.36366
[2000]	valid_0's quantile: 1.35569
Did not meet early stopping. Best iteration is:
[2000]	valid_0's quantile: 1.35569
0.1   ###################################### 1296개 중 366개 선정 완료 ##############################################
[LightGBM] [Warning] Unknown parameter: Min_data_in_leaf
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] 

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3883,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3884,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
3885,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
3886,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
print('소요시간 : ', (tock - tick)/60, '분')
print(results_1.shape, results_2.shape)
#print('점수 : ', rnd_search.best_score_)
#print('최적 파라미터 : ', rnd_search.best_params_)

소요시간 :  152.88923313617707 분
(3888, 9) (3888, 9)


In [108]:
from scipy.optimize import curve_fit
import pylab
import math

# columns_IDA는 상하로 맞춰줌
# rows_IDA는 좌우로 맞춰줌
# 상하 > 좌우 & 좌우 > 상하를 비교한 결과 좌우로 먼저 맞춘 다음에 상하를 맞추는 것이 쬐------끔 좋은 결과를 보임.

# 그래서 rows_IDA 먼저 하고 columns_IDA 추천


def columns_IDA(dataset):
  dataset2 = pd.DataFrame()
  for j in range(0,9):
    new_val = []
    for m in range(1, int(raw_data.shape[0]/48)+1):
      temp = dataset.iloc[48*(m-1):48*m,j].values

      strt = 0
      endd = 0
      k = 0

      for i in range(len(temp)):
        if temp[i] > k and k ==0 and i < 24:
          strt = i
        elif temp[i] < k and temp[i] == 0 and i > 24:
          endd = i
        k = temp[i]
      if strt != 0:
        bf = temp[:strt+1]
        y = temp[strt+1:endd-1]
        af = temp[endd-1:]
        # bf = temp[:strt]
        # y = temp[strt:endd]
        # af = temp[endd:]

        x = np.array([i for i in range(len(y))])
        fit_t = np.polyfit(x,y,5)

        for i in x:
          fit1 = fit_t[0]*x**5 + fit_t[1]*x**4 + fit_t[2]*x**3 + fit_t[3]*x**2 + fit_t[4]*x + fit_t[5]
      # fit1 = fit_t[0]*x**4 + fit_t[1]*x**3 + fit_t[2]*x**2 + fit_t[3]*x + fit_t[4]
      # fit1 = fit_t[0]*x**2 + fit_t[1]*x + fit_t[2]

        revised = np.concatenate((bf, fit1, af), axis=0)
      elif strt == 0:
        revised = temp
      new_val.extend(list(revised))
    k = pd.DataFrame(new_val)
    dataset2 = pd.concat([dataset2, k], axis = 1)
  return dataset2

def f(x, a,b,c, d):
  y = 1/3 * b * x**3 - a*b *x**2 + (a**2*b + b*c**2)*x + d
  # y = a*(x-12*a) * x **2 + c
  # y = a*x**2 + b
  return y

def rows_IDA(dataset):
  revised = pd.DataFrame()
  for i in range(dataset.shape[0]):
    y = dataset.iloc[i,:].values
    x = np.array([i for i in range(1, 10)])

    if list(y).count(0) <= 6 :
      popt, pcov = curve_fit(f, x, y, maxfev = 999990000)
      yfit = f(x, *popt)
      revised = pd.concat([revised, pd.DataFrame(yfit).transpose()], axis = 0)
    else:
      revised = pd.concat([revised, pd.DataFrame(y).transpose()], axis = 0)
    if i % 1000 == 0:
      print(i, '/7776')
  return revised

In [166]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [167]:
dataset2 = rows_IDA(submission3)
dataset3 = pd.DataFrame(dataset2.values, columns = submission3.columns, index = submission3.index)
dataset3 = not_minus(dataset3)
dataset4 = columns_IDA(dataset3)
dataset5 = pd.DataFrame(dataset4.values, columns = submission3.columns, index = submission3.index)
submission3 = not_minus(dataset5)
dataset2 = rows_IDA(submission3)
dataset3 = pd.DataFrame(dataset2.values, columns = submission3.columns, index = submission3.index)
submission3 = not_minus(dataset3)
submission3 = not_dec(submission3)

0 /7776


/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


1000 /7776
2000 /7776
3000 /7776
4000 /7776
5000 /7776
6000 /7776
7000 /7776
0 /7776
1000 /7776
2000 /7776
3000 /7776
4000 /7776
5000 /7776
6000 /7776
7000 /7776


In [168]:
submission4 = pd.DataFrame(submission3.values, columns = submission2.columns, index = submission2.index)

In [169]:
submission4.to_csv('Feature_selection.csv')
!cp Feature_selection.csv "drive/My Drive/"